<a href="https://colab.research.google.com/github/ethanchuicy/insti_resistance_model/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import dataset
import pandas as pd
col_names = []
df = pd.read_csv('/integrase_dataset', sep='\t')
print(df)

In [ ]:
df = df.drop(columns = ['RefID', 'IsolateID', 'IsolateName', 'Species', 'Type', 'PtID', 'Method'])
print(df.head())

In [ ]:
df1= df.drop(columns = ['BICFoldMatch', 'CABFoldMatch', 'DTGFoldMatch', 'EVGFoldMatch', 'RALFoldMatch', 'CompleteMutationListAvailable', 'Author'])
print(df1.head())

In [ ]:
df2= df1.drop(columns=['INIMajorDRMs', 'INIMinorDRMs', 'NonDRMs', 'RefYear', 'MedlineID'])
print(df2.head())

In [ ]:
df3= df2.drop(columns=['Title'])
print(df3.head())

In [ ]:
all_pos_columns = [f'P{i}' for i in range(1, 289)]
df3[all_pos_columns] = df3[all_pos_columns].replace('-', 0)
print(df3.head())

In [ ]:
#Apply one-hot encoding
encoded_df = pd.get_dummies(df3[all_pos_columns])


df4 = df3.drop(all_pos_columns, axis=1)
df5 = pd.concat([df4, encoded_df], axis=1)


print(df5)

In [ ]:
df5['EVG'] = pd.to_numeric(df5['EVG'], errors='coerce')


In [ ]:
df5['RAL'] = pd.to_numeric(df5['RAL'], errors='coerce')

In [ ]:
df_filled = df5.fillna(-999)
bic_list = df_filled['BIC'].to_list()
cab_list = df_filled['CAB'].to_list()
dtg_list = df_filled['DTG'].to_list()
evg_list = df_filled['EVG'].to_list()
ral_list = df_filled['RAL'].to_list()

resistance= []

for i in range(0, 1103):
  if bic_list[i]>3.5:
    resistance.append(1)
  elif cab_list[i]>3.5:
    resistance.append(1)
  elif dtg_list[i]>3.5:
    resistance.append(1)
  elif evg_list[i]>3.5:
    resistance.append(1)
  elif ral_list[i]>3.5:
    resistance.append(1)
  else:
    resistance.append(0)

print(resistance)
print(len(resistance))

In [ ]:
df5['Resistance']= resistance
print(df5.head())

In [ ]:
columns_to_exclude = ['BIC', 'CAB', 'DTG', 'EVG', 'RAL', 'Resistance']
df_selected = df5.drop(columns=columns_to_exclude)
X = df_selected
y= df5['Resistance']

In [ ]:
#Divide dataset into k-fold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
#log regression model
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression

my_lambda = 10
c = 1/my_lambda
model = LogisticRegression(solver='liblinear', penalty='l1', C=c)
score = cross_val_score(model, X, y, cv=cv, n_jobs=-1)

print(score)
print(mean(score))
print(std(score))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
final_model = model.fit(X_train, y_train)
prediction = final_model.predict(X_test)
#metrics
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, prediction)


In [ ]:
#Gert coefficients
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
print(final_model.coef_)

In [ ]:
coef_list = final_model.coef_.tolist()
index_list = []

for i in range(len(coef_list[0])):
  if coef_list[0][i] != 0:
    index_list.append(i)


print(index_list)

col_names = X.columns.values.tolist()
print(col_names[466])

print(coef_list[0][466])

for i in index_list:
  print(col_names[i])
  print(coef_list[0][i])





In [ ]:
#classification report
from sklearn.metrics import classification_report
target_names = ['Not Resistant', 'Resistant']
print(classification_report(y_test, prediction, target_names=target_names))

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class_names=['Not Resistant', 'Resistant'] # name  of classes
fig, ax = plt.subplots()
# create confusion matrix
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g', annot_kws={"size": 14, "weight": "bold"})
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.ylabel('Actual label', fontsize=12, fontweight="bold")
plt.xlabel('Predicted label', fontsize=12, fontweight="bold")
ax.set_xticklabels(class_names, fontsize=12, fontweight="bold" )
ax.set_yticklabels(class_names, fontsize=12, fontweight="bold" )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=12, labelcolor='black')  # Tick size and color
for label in cbar.ax.get_yticklabels():
    label.set_fontweight("bold")



In [ ]:
#roc curve
y_pred_proba = model.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="AUC="+str(round(auc, 2)))
plt.legend(loc=4)
plt.title('ROC Curve for Final Model')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()